# Import Statements

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from matplotlib import pyplot as plt
import dominate
from dominate.tags import *
import espn_api.football
from espn_api.football import League
import time
import pickle
import uuid

from python import constants, functions

# Testing and Debugging

In [ ]:
def content_container(
        title: str,
        content: div | table | img,

        filter_id: str = None,
        filter_column: pd.Series = None,
        filter_showall: bool = False
) -> div:
    '''
    Creates a div with class "content-container," including an h2 title tag and content. Optionally may include a dropdown filter if content is a table.

    Parameters
    ----------
    title : str
        Name to be set as the title for the content bubble.

    content : div | table | img
        General content that will be added to the body of the bubble. May be any of the following:
            * div
            * table
            * img

    filter_id : str, default None
        ID to be given to dropdown filter.

    filter_column : pd.Series, default None
        Series from the DataFrame used for content table. Should always be set to the first column in row_id_columns argument in df_to_table().
    
    filter_showall : bool, default False
        Choose whether filter dropdown features All as an option.

    Returns
    -------
    div
        Return a dominate div object. 
    '''
    container = div(_class='content-container')
    container_title = h2(title)

    if filter_id:
        container_select_options = [option(record, value=str(record).lower().replace(' ','')) for record in filter_column.unique()]
        if filter_showall:
            container_select_options.insert(0, option('All', value='all'))

        if type(content) == table:
            table_id = content['id']
        else:
            table_id = content.get(table)[0]['id']
        container_select = select(container_select_options,
                                  _id=filter_id,
                                  onchange=f"tableFilter('{filter_id}', '{table_id}')")
        container_title.add(container_select)

    container_content = content

    container.add(container_title, container_content)

    return container

In [18]:
year = 2023
week = 3

lineup_data = constants.PLAYER_MATCHUP_DATA.loc[(constants.PLAYER_MATCHUP_DATA['Year'] == year) & (constants.PLAYER_MATCHUP_DATA['Week'] == week)].copy()
lineup_data['Matchup Lookup'] = lineup_data['Home Team'].astype(str) + ' vs ' + lineup_data['Away Team'].astype(str)
lineup_table = functions.df_to_table(
    data=lineup_data,
    custom_columns=['Home Team','Home Player','Home Player Points','Position','Away Player Points','Away Player','Away Team'],
    row_id_columns=['Matchup Lookup','Position'],
    table_id='lineup2-table'
)
lineup_div = content_container(
    title='Lineups',
    content=lineup_table,
    filter_id='lineup2-filter',
    filter_column=lineup_data['Matchup Lookup'],
    filter_showall=False
)


print(lineup_div)

<div class="content-container">
  <h2>Lineups
    <select id="lineup2-filter" onchange="tableFilter('lineup2-filter', 'lineup2-table')">
      <option value="cartervsandrew">Carter vs Andrew</option>
      <option value="dantevszach">Dante vs Zach</option>
      <option value="harisvsnathan">Haris vs Nathan</option>
      <option value="kevinvsethan">Kevin vs Ethan</option>
      <option value="mcgwirevsnoah">McGwire vs Noah</option>
      <option value="michaelvstyler">Michael vs Tyler</option>
    </select>
  </h2>
  <div class="scroll-table">
    <table id="lineup2-table">
      <thead>
        <tr>
          <th class="home-team">Home Team</th>
          <th class="home-player">Home Player</th>
          <th class="home-player-points">Home Player Points</th>
          <th class="position">Position</th>
          <th class="away-player-points">Away Player Points</th>
          <th class="away-player">Away Player</th>
          <th class="away-team">Away Team</th>
        </tr>
     